### 记录torch操作

感觉在两个算例里面最头疼的还是tensor的这些操作，在实例中看感觉还蛮清晰的，读代码就有点一头雾水

---


**torch.nn.Linear**

```python
# 参数如下 只需要给定一组输入中输入的feature数和输出的feature数
nn.Linear(in_features = 8, out_features = 16)
```

在写CNN的时候产生的一些困惑，关于batch和这个Linear层

Linear层相当于一个矩阵M，形状为[input,output],

那我们假定输入为X，[batch_size,input]，输出为Y，[batch_size,output]

就有$Y = X \cdot M + b$

所以想要表达的是，batch的存在只是对输出增加了一个重复堆叠的过程，对Linear的参数毫无影响

---


**view操作**

是对张量的一个重构操作，只需明白存储是从里往外一行一行存，重构时的填充是从内往外一行一行填

In [ ]:
import torch

x = torch.tensor([[ 1.,  2.,  3.,  1.,  2.,  3.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  4.,  5.,  6.,  4.,  5.,  6.],
        [ 7.,  8.,  9.,  7.,  8.,  9.,  7.,  8.,  9.],
        [10., 11., 12., 10., 11., 12., 10., 11., 12.],
        [13., 14., 15., 13., 14., 15., 13., 14., 15.]])
print(x)

x = x.view(5 ,3 ,3)
print(x)
x = x.view(3 ,5 ,3)
print(x)


---

**permute操作**

对原始张量shape的换序

In [ ]:
print(x.shape)
x = x.permute(0, 2, 1)
print(x.shape)


---

**torchvision.transforms.compose类**

感觉是实现了spatial transformer一层的功能，compose将几个预处理的函数串联在一起，但是用法不太直观，首先说明用法

In [9]:
import torchvision.transforms as transforms
from PIL import Image

tranform = transforms.Compose([
    transforms.Resize((128, 128)),
    # 顾名思义
    transforms.ToTensor(),
    # img转成张量，RGB值归一化到01之间
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # 归一化
])

# img = Image.open('img.jpg')
# img_tensor = tranform(img)

可以注意到compose里面函数给的参数也比较神秘，看不到img的影子，实际上是compose作用于img时已经把img传进内部的几个函数里面。
另外可以看到，transform作为compose类的实例化，使用时transform(img)与函数的用法一样，是使用了__call__方法，也是比较神奇
这两点具体还是得把compose的原理写一写

In [10]:
class compose:
    
    def __init__(self, functions):
        self.functions = functions

    def __call__(self, img):
        for function in self.functions:
            img = function(img)
        return img


---

**torch.utils.data中的Dataset&Dataloader**

假定我们现在只有X和Y（即feature和label），torch提供一个抽象类Dataset，继承这个抽象类，并赋予它self.X,self.Y,以及__getitem__,__len__两个方法，最后用我们的XY实例化出一个dataset。
要想访问这个datset，使用torch提供的Dataloader类，实例化一个dataloader，可以分批取出data，dataloader本身是一个可迭代对象，可以通过下面的方式取出每一个batch的x，y。
``` python
for x, y in dataloader
```
如果想得到每个batch的index也可以用enumerate的方式。
``` python
for x, y in enumerate(dataloader)
```

In [5]:
import torch
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
    def __init__(self, X, Y):
        super().__init__()
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        x = torch.tensor(self.X[index])
        y = torch.tensor(self.Y[index])
        return (x, y)
    
    def __len__(self):
        return len(self.X)

dataset = MyDataset([1,2,3,4,5,6], [10,20,30,40,50,60])
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

for x, y in dataloader:
    print(x,y)

for x, y in enumerate(dataloader):
    print(x,y)

tensor([1, 2]) tensor([10, 20])
tensor([3, 4]) tensor([30, 40])
tensor([5, 6]) tensor([50, 60])
0 [tensor([1, 2]), tensor([10, 20])]
1 [tensor([3, 4]), tensor([30, 40])]
2 [tensor([5, 6]), tensor([50, 60])]
